In [1]:
from readers.ria_reader import ria_reader
from datasets.gen_title_dataset import GenTitleDataset
from models.bottleneck_encoder_decoder import BottleneckEncoderDecoderModel
from transformers import AutoTokenizer, EncoderDecoderModel, PreTrainedModel, PretrainedConfig, \
    Trainer, TrainingArguments, logging

In [2]:
import json
import random

In [3]:
from _jsonnet import evaluate_file as jsonnet_evaluate_file

In [4]:
config = json.loads(jsonnet_evaluate_file('configs/gen_title.jsonnet'))

In [5]:
test_file = '/Users/leshanbog/Documents/dataset/ria/lil_ria.json'
test_sample_rate = 1.0
model_file = 'models/gen_title'
enable_bottleneck = False

In [6]:
print("Fetching data...")
test_records = [r for r in ria_reader(test_file) if random.random() <= test_sample_rate]

Fetching data...


In [7]:
print("Building datasets...")
model_path = config.pop("model_path")
tokenizer = AutoTokenizer.from_pretrained(model_path, do_lower_case=False, do_basic_tokenize=False)

# max_tokens_text = config.pop("max_tokens_text", 196)
# max_tokens_title = config.pop("max_tokens_title", 48)

Building datasets...


In [8]:
a = test_records[0]['title']
a

'тренер фк "спартак" сетует на реализацию моментов в игре с "томью"'

In [9]:
txt = tokenizer(a, verbose=3)
txt

{'input_ids': [101, 10109, 881, 862, 108, 37111, 1533, 108, 9509, 4742, 1469, 36392, 43296, 845, 13240, 869, 108, 3835, 3453, 108, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
tokenizer.decode(txt['input_ids'][1:-1])

'тренер фк " спартак " сетует на реализацию моментов в игре с " томью "'

In [ ]:
test_dataset = GenTitleDataset(
    test_records,
    tokenizer,
    max_tokens_text=max_tokens_text,
    max_tokens_title=max_tokens_title
)

In [11]:
from transformers import EncoderDecoderConfig


In [12]:
model_config = EncoderDecoderConfig.from_pretrained(model_file)
model = EncoderDecoderModel.from_pretrained(model_file, config=model_config)
model.eval()

In [13]:
model.eval()

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [20]:
data = test_dataset[4].copy()
data.pop('labels')

tensor([   101,  11808,  16860, 100532,   1755,  15042,   7239,   2190,    845,
        109909,  59685,   4161,    102,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100])

In [21]:
data

{'input_ids': tensor([   101, 109909,  29414,    128,    144,   2862,    848,    130,  66385,
            626,  28338,    132,  12844,  62131,    858,    130,  60756,   1388,
           1516,  88921,  16842,    869,  28405, 100532,   1755,    851,  55389,
          16135,   9058,   5022,  95450,    845, 109909,  59685,   4161,    845,
           3491,   2077,    128,   9880,  20616,    845,  14640,  13353,  14147,
         109909,  36332,  10212,  33266,   7009,  23291,  11907,   9200,    132,
           1516,   2752,   6335,    128,  13586,  33221,   3952,  19782,  62131,
            858,    130,  60756,    880,    130,  61625,   2059,   9599,   1469,
           5296,    130,   5848,  69727,  19531,   2785,  11173,  47455,    130,
          27154,  22385,   2785,  10762,   4638,   1768,    845, 109909,  75669,
            128,  45097,   3005,    128,  15343,   6619,   3005,    128, 102109,
           3005,    128,   4692,   5176,   3005,    128,  25404,  41874,   9127,
            130

In [25]:
data['input_ids']

tensor([   101, 109909,  29414,    128,    144,   2862,    848,    130,  66385,
           626,  28338,    132,  12844,  62131,    858,    130,  60756,   1388,
          1516,  88921,  16842,    869,  28405, 100532,   1755,    851,  55389,
         16135,   9058,   5022,  95450,    845, 109909,  59685,   4161,    845,
          3491,   2077,    128,   9880,  20616,    845,  14640,  13353,  14147,
        109909,  36332,  10212,  33266,   7009,  23291,  11907,   9200,    132,
          1516,   2752,   6335,    128,  13586,  33221,   3952,  19782,  62131,
           858,    130,  60756,    880,    130,  61625,   2059,   9599,   1469,
          5296,    130,   5848,  69727,  19531,   2785,  11173,  47455,    130,
         27154,  22385,   2785,  10762,   4638,   1768,    845, 109909,  75669,
           128,  45097,   3005,    128,  15343,   6619,   3005,    128, 102109,
          3005,    128,   4692,   5176,   3005,    128,  25404,  41874,   9127,
           130,  14024,   3005,    851, 

In [29]:
data['input_ids'].unsqueeze(0)

torch.Size([1, 196])

In [80]:
import torch

In [89]:
torch.cat((data['input_ids'].unsqueeze(0), data['input_ids'].unsqueeze(0)), dim=0).shape

torch.Size([2, 196])

In [124]:
out_file = 'ria.pred'
batch_size = 4

with open(out_file, 'w', encoding='utf-8') as f:
    for i in range(0, len(test_dataset), batch_size):
        data = test_dataset[i]
        del data['labels']
        
        for k in data.keys():
            data[k] = data[k].unsqueeze(0)
        
        for j in range(i + 1, min(i + 4, len(test_dataset))):
            for k in data.keys():
                data[k] = torch.cat((data[k],
                                     test_dataset[j][k].unsqueeze(0)), dim=0)
        

        output_ids = model.generate(**data,
                                    decoder_start_token_id=model.config.decoder.pad_token_id)
        preds = [
            tokenizer.decode(x[1: torch.max(torch.nonzero(x)).item()]) for x in output_ids
        ]
        
        f.write('\n'.join(preds))

In [125]:
cat ria.pred

тренер фк " спартак " сетует на реализацию моментов в игре с " томью "
нина павловна гребешкова. биографическая справка
подозрительный предмет нашли в " манеже " в москве, сообщил источник
обвиняемый в организации беспорядков в ереване направлен на экспертизупервые десять энергоэффективных домов построят в ивановской области
события южного и северо - кавказского федеральных округов 15 марта
россия могла продать египту средства пво более чем на $ 2 млрд
не менее шести человек погибли в результате теракта в кабулекруглый стол " инновации в спорте "
рынок акций рф открылся разнонаправленно на внешнем негативе
" бахослужение " откроется в калининградской области
ведущие страховщики рф показали в i полугодии значительный рост премийбелгородские школьники отправили сладости выпускникам, ушедшим в армию
ростовским министром труда стала глава службы занятости региона
пламя высотой 10 метров поднималось на месте взрыва газопровода в польше
минобороны подготовилось к обеспечению военных жильем ч